In [38]:
import numpy as np
import math
import time
import datetime
import cv2
import imutils

# Mejoras: leer video desde un segundo dado

In [2]:
imutils.find_function("contour")

1. CONTOURS_MATCH_I1
2. CONTOURS_MATCH_I2
3. CONTOURS_MATCH_I3
4. contourArea
5. drawContours
6. findContours
7. isContourConvex
8. ximgproc_ContourFitting


In [3]:
def Calcular_Numero(t0, tr0, tcorrido, Pin_Dom):
    ind = 0
    print(t0, tr0, tcorrido)
    b = math.sqrt(Beta/alfa)
    #print ("b: ", b)
    x = ((math.pow(np.e, alfa * 2 * 3.1415) - math.cosh(alfa * b * t0))/math.sinh(alfa * b * t0))
    #print ("x; ", x)
    c1 = math.pow(b,2)*(math.pow(x,2)-1)
    #print ("c1: ", c1)
    distancia_fallo = (1/(2 * alfa)) * math.log(c1 / (math.pow(velocidad_fallo,2) - math.pow(b,2)))
    c0 = -math.pow(np.arcsinh(b/math.sqrt(c1)),1)
    #print ("c0: ",c0)
    tiempo_fallo = (1/(alfa * b)) * (c0 - np.arcsinh(math.sinh(c0) * math.pow(np.e,(alfa * distancia_fallo))))
    print("tiempo fallo: ", tiempo_fallo)
    distancia_fallo = (1/(2 * alfa)) * math.log( np.sinh(c0 - (alfa * b * tiempo_fallo)) / np.sinh(c0))
    print ("distancia bola  fallo: ", distancia_fallo)
    circunferencia = 2 * 3.1415 * r 
    #print ("circunferencia: ", circunferencia)
    distancia_fallo_rps = (distancia_fallo / circunferencia) 
    #   print ("distancia bola fallo (rps): ", distancia_fallo_rps)
    distancia_rotor_fallo = (tiempo_fallo - t0 - tcorrido) / (tr0 * 1)
    print("vueltas rotor fallo: ", distancia_rotor_fallo)
    posicion_cero = 37 - (distancia_rotor_fallo - (int)(distancia_rotor_fallo))*37
    print("Posicion Cero: ", posicion_cero)
    #print("Pin Dom: ", Pin_Dom)
    if (Pin_Dom == 4):
        ind = (int)(posicion_cero)
        numero_golpea = numeros_ruleta[(int)(posicion_cero)] 

    if (Pin_Dom == 3):
        if ((int)(posicion_cero)+9 > 36):
            ind = (int)(posicion_cero)+9-36
            numero_golpea = numeros_ruleta[(int)(posicion_cero)+9-36] 
        else:
            ind = (int)(posicion_cero)+9
            numero_golpea = numeros_ruleta[(int)(posicion_cero)+9]
    
    if (Pin_Dom == 2):
        if ((int)(posicion_cero)+18 > 36):
            ind = (int)(posicion_cero)+18-36
            numero_golpea = numeros_ruleta[(int)(posicion_cero)+18-36] 
        else:
            ind = (int)(posicion_cero)+18
            numero_golpea = numeros_ruleta[(int)(posicion_cero)+18] 
    
    if (Pin_Dom == 1):
        if ((int)(posicion_cero)+27 > 36):
            ind = (int)(posicion_cero)+27-36
            numero_golpea = numeros_ruleta[(int)(posicion_cero)+27-36] 
        else:
            ind = (int)(posicion_cero)+27
            numero_golpea = numeros_ruleta[(int)(posicion_cero)+27] 

    print("Indice Golpea: ", ind)
    print("Numero Golpea: ", numero_golpea)

    if ((ind-rebote) <0):
        numero_ganador = numeros_ruleta[ind-rebote+36]
    else:
        numero_ganador = numeros_ruleta[ind-rebote]

    print("Numero Ganador: ", numero_ganador)

In [4]:
numeros_ruleta = [0, 26, 3, 35, 12, 28, 7, 29, 18, 22, 9, 31, 14, 20, 1, 33, 16, 24, 5, 10, 23, 8, 30, 11, 36, 13, 27, 6, 34, 17, 25, 2, 21, 4, 19, 15, 32]


In [60]:
class Ruleta:
  
  def __init__(self, numbers):
    self.numbers = numbers
    self.parameters={}
  
  def setParameters(self,params):
    self.parameters=params
    
  def showNumbers(self):
    print(self.numbers)

  def setPinColorLimit(self):
    self.pinLower = (105, 105, 105)
    self.pinUpper = (140, 140, 140)
    
  def setZeroColorLimit(self):
    self.ceroLower = (25, 45, 30)
    self.ceroUpper = (35, 55, 40)
    
  def setBallColorLimit(self):
    self.ballLower = (50, 50, 50)
    self.ballUpper = (55, 55, 55)    
    
  def setInitFrameValues(self):
    self.frames = 0
    self.frames_ant = 0
    self.vueltas = -1
    self.flag_vuelta = 0
    self.sumatoria = 0
    self.vueltas_rotor = -1
    self.frames_rotor_ant = 0
    self.frame_pos = 0
    self.tiempo_corrido = 0
    self.flag_calculo = 0
        
    
  def loadVideo(self,video):
    self.cap = cv2.VideoCapture(video)
    self.fps=self.cap.get(cv2.CAP_PROP_FPS)
    self.frame_count = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print("fps: ",self.fps)  
    print("frame_count: ",self.frame_count)  
    print("Duration: ",(self.frame_count/self.fps) / 60," min")  
    
  def printTime(self,frame):
    font = cv2.FONT_HERSHEY_SIMPLEX
    datet = str(datetime.datetime.now())
    frame = cv2.putText(frame, datet, (10, 30), font, 1,(0, 255, 255), 1, cv2.LINE_AA)
    
  def skipFrames(self,numbers):
    while self.i < numbers:
      self.ret, self.frame = self.cap.read() 
      self.frames = self.frames + 1
      self.i = self.i + 1;
    
  def createMask(self):
    self.maskpin  = cv2.inRange(self.frame_aux, self.pinLower, self.pinUpper)
    self.maskcero = cv2.inRange(self.frame_aux, self.ceroLower,self.ceroUpper)
    self.maskbola = cv2.inRange(self.frame_aux, self.ballLower,self.ballUpper)

  def findContours(self):
    test,contour,cntspin = cv2.findContours(self.maskpin.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    cntspin = imutils.grab_contours(contour)
    if (len(cntspin)>0):
        ymenor = 1000
        xmenor = 0
        for j, contour in enumerate(cntspin, start=1):
            sh = contour.shape[0]
            for k in range(sh-1):
                xpcontour = contour[k][0][0]
                ypcontour = contour[k][0][1] 
                if (ypcontour < ymenor):
                    ymenor = ypcontour
                    xmenor = xpcontour
            
        cv2.circle(self.frame_aux, (xmenor, ymenor+20), 1, (0, 0, 255), 10)
        cv2.circle(self.frame_aux, (xmenor, ymenor+80), 1, (0, 255, 255), 10)
        cv2.circle(self.frame_aux, (xmenor, ymenor+120), 1, (0, 255, 255), 10)
        
  def findBallContours(self):
    cntsbola = cv2.findContours(self.maskbola.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    cntsbola = imutils.grab_contours(cntsbola)

    if (len(cntsbola)>0):
      cv2.circle(self.frame_aux, (300,67), 10, (0, 255, 255), 2)
      for j, contour in enumerate(cntsbola, start=1):
        sh = contour.shape[0]
        pasobola = 0
        for k in range(sh-1):
          xcontour = contour[k][0][0]  
          ycontour = contour[k][0][1] 
          cv2.circle(self.frame_aux, (xcontour,ycontour), 1, (255, 255, 255), 2)
          #Si paso por marca es nueva vuelta
          if ((xcontour >= 295 and xcontour <= 305) and
                  (ycontour >= 55 and ycontour <= 75) and 
                      (self.frames_ant + 10 < self.frames) ):
            pasobola = 1;
            break;
    
          if ( pasobola > 0):            
            self.vueltas = self.vueltas + 1
            if (self.vueltas == 0):   
              print("Vuelta 0")
              self.frames_ant = self.frames
              break;
            if (self.vueltas > 0):
              #Posicion del Cero al cumplir Vuelta 1
              if (self.vueltas == 1):
                self.flag_vuelta = 1;
                self.frame_pos = self.frames;
                print("Pos Rotor Vuelta 1")
                self.t0 = (self.frames-self.frames_ant)/25
                time.sleep(3.5)
                self.sumatoria = self.sumatoria +  (self.frames-self.frames_ant)/25
                velocidad = 1/((self.frames-self.frames_ant)/25)
                print("Frame: ", self.frames, " Vuelta:", self.vueltas, "Tiempo x Vuelta ", 
                      (self.frames-self.frames_ant)/25, "Velocidad x Vuelta ", velocidad)
                self.frames_ant = self.frames;
                break;
                
  def findZeroContours(self):
    cntscero = cv2.findContours(self.maskcero.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    cntscero = imutils.grab_contours(cntscero)
    if (len(cntscero)>0):
        cv2.circle(self.frame_aux, (370,110), 10, (0, 255, 255), 2)
        cv2.circle(self.frame_aux, (370,150), 10, (0, 255, 255), 2)
        for l, contour in enumerate(cntscero, start=1):
            sh = contour.shape[0]
            pasocero = 0
            for k in range(sh-1):
                xrcontour = contour[k][0][0]
                yrcontour = contour[k][0][1]
                cv2.circle(self.frame_aux, (xrcontour,yrcontour), 1, (255, 255, 0), 2)
                if ((xrcontour >= 320 and xrcontour <= 420) and 
                     (yrcontour >= 90 and yrcontour <= 170) and
                      self.frames_rotor_ant + 10 < self.frames and self.flag_vuelta == 1 ):
                    print("Si Cero")
                    pasocero = 1;
                    break;
            
            if (pasocero > 0):   
                self.vueltas_rotor = self.vueltas_rotor + 1
                self.tiempo_corrido = (self.frames - self.frame_pos)/25   
                if (self.vueltas_rotor == 0):   
                    print("Vuelta Rotor 0")
                    self.tiempo_corrido = (self.frames - self.frame_pos)/25    
                    print("Frame: ", self.frames, "Frame Pos: ", self.frame_pos, "Tiempo Corrido: ", self.tiempo_corrido)
                    self.frames_rotor_ant = self.frames
                    break;
            
                if (self.vueltas_rotor > 0):
                    self.tr0 = ((self.frames-self.frames_rotor_ant)/25)
                    velocidad = 1/((self.frames-self.frames_rotor_ant)/25)
                    print("Frame: ", self.frames, "Frame Rotor Ant", self.frames_rotor_ant, " Vuelta Rotor: ", vueltas_rotor, "Tiempo Rotor x Vuelta ", (self.frames-self.frames_rotor_ant)/25, "Velocidad Rotor x Vuelta", velocidad)
                    self.frames_rotor_ant = self.frames;
                    break;    
                    
  def manageUserInput(self,key):
    if key == ord("a"):
      self.framea = self.frames

    if key == ord("s"):
      print("Tiempo Cron: ", (self.frames-self.framea)/25)

    if key == ord("t"):
      print("Tiempo Parcial: ", self.sumatoria)
      self.sumatoria = self.sumatoria +  (self.frames-self.frames_ant)/25
      print("Frame: ", self.frames, " Vuelta : ", "Final", "Tiempo x Vuelta ", (self.frames-self.frames_ant)/25)
      print("Tiempo Total: ", self.sumatoria)

    if key == ord("r"):
      self.setInitFrameValues()

    if key == ord("o"):
      time.sleep(30.5)
  
    
  def play(self,init=0):
    self.i = 0
    self.setInitFrameValues()
    #self.skipFrames(6400)
    self.cap.set(1,10);

    while True:
      ret, frame = self.cap.read() 
      self.frames = self.frames + 1
      if frame is None:
        break
        
      self.frame_aux = imutils.resize(frame, width=800)  
      self.createMask()
      
      #self.findContours()
      #self.findBallContours()
      #self.findZeroContours()
      if (self.vueltas >= 1 and self.vueltas_rotor >=1 and self.flag_calculo == 0):
        Calcular_Numero(self.t0, self.tr0, self.tiempo_corrido, self.parameters.pin_dom)
        self.flag_calculo = 1;
        
      key = cv2.waitKey(1) & 0xFF  
      self.manageUserInput(key)  
        
      if key == ord("q"):
        break  
        
      self.printTime(self.frame_aux)  
      cv2.imshow("Framer", self.frame_aux)
        
    self.cap.release()
    cv2.destroyAllWindows()
      


In [62]:
ruleta_parameters={
  "alfa" : 0.026,
  "r" : 0.375,
  "beta" : 0.075,
  "velocidad_fallo" : 3,
  "rebote" : 7,
  "pin_dom" : 3
}

numeros_ruleta = [0, 26, 3, 35, 12, 28, 7, 29, 18, 22, 9, 31, 14, 20, 1, 33, 16, 24, 5, 10, 23, 8, 30, 11, 36, 13, 27, 6, 34, 17, 25, 2, 21, 4, 19, 15, 32]

ruleta =Ruleta(numeros_ruleta)
ruleta.setParameters(ruleta_parameters)
ruleta.setPinColorLimit()
ruleta.setZeroColorLimit()
ruleta.setBallColorLimit()

ruleta.loadVideo('C:/ruleta/Jafco.mp4')
ruleta.play()


fps:  25.000732229224916
frame_count:  34210
Duration:  22.805998697916667  min
